In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2 as cv
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.utils import Sequence
from tensorflow.keras import layers, models

In [2]:
train_dataGen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)
train_src = r'UCF_crime_dataset\Train'
test_src = r'UCF_crime_dataset\Test'

In [44]:
class VideoGen(Sequence):
    def __init__(self, dir, sequence_frame, target_size, batch_size, class_mdoe = 'sprase'):
        self.directory = dir
        self.no_of_frame = sequence_frame
        self.batch_size = batch_size
        self.img_dimensions = target_size
        self.class_representation = class_mdoe
        self.classes = sorted(os.listdir(dir))
        self.class_indices = {cls: idx for idx, cls in enumerate(self.classes)}
        self.samples = self._get_all_samples()
        self.on_epoch_end()

    def _get_all_samples(self):
        samples = []
        for class_name in self.classes:
            class_dir = os.path.join(self.directory, class_name)
            frame_file = sorted(os.listdir(class_dir))
            for i in range(len(frame_file) - self.no_of_frame + 1):
                sample = (class_dir, i, class_name)
                samples.append(sample)
        return samples
    
    def __len__(self):
         return len(self.samples) // self.batch_size
    
    def __getitem__(self, index):
        batch_samples = self.samples[index * self.batch_size:(index + 1) * self.batch_size]
        X, y = self._load_batch(batch_samples)
        return X, y

    def _load_batch(self, batch_samples):
        X = np.empty((self.batch_size, self.no_of_frame, *self.img_dimensions, 3))
        y = np.empty((self.batch_size,), dtype=int)

        for i, (class_dir, start_idx, class_name) in enumerate(batch_samples):
            for j in range(self.no_of_frame):
                frame_path = os.path.join(class_dir, sorted(os.listdir(class_dir))[start_idx + j])
                image = load_img(frame_path, target_size=self.img_dimensions)
                X[i, j] = img_to_array(image) / 255.0

            y[i] = self.class_indices[class_name]

        return X, y

    def on_epoch_end(self):
        np.random.shuffle(self.samples)


In [45]:
batch_szie = 32
img_dimension = (64, 64)
no_of_frame = 10
class_mode = 'sparse'

train_dataGen = VideoGen(
    dir = train_src,
    sequence_frame = no_of_frame,
    target_size = img_dimension,
    batch_size = batch_szie,
    class_mdoe = class_mode
)


test_datagen = VideoGen(
    dir = test_src, 
    target_size = img_dimension,
    sequence_frame = no_of_frame, 
    batch_size = batch_szie,
    class_mdoe = class_mode
)

In [46]:
class conv2dPlus1d(layers.Layer):
    def __init__(self, filters, kernel_size, padding = 'same', activation = 'relu'):
        super(conv2dPlus1d, self).__init__()
        self.seq = models.Sequential([

            #spacial infomation
            layers.Conv3D(filters = filters,
                          kernel_size = (1, kernel_size[1], kernel_size[2]), 
                          activation = activation,
                          padding = padding),
            
            #temporal information
            layers.Conv3D(filters = filters,
                          kernel_size = (kernel_size[0], 1, 1),
                          padding = padding,
                          activation = activation)
        ])

    def call(self, x):
        return self.seq(x)

In [47]:
def build_model(input_shape):
    model = models.Sequential([
        layers.Input(shape=input_shape),
        conv2dPlus1d(filters=32, kernel_size=(3, 3, 3), padding='same', activation='relu'),
        layers.MaxPooling3D(pool_size=(1, 2, 2)),
        conv2dPlus1d(filters=64, kernel_size=(3, 3, 3), padding='same', activation='relu'),
        layers.MaxPooling3D(pool_size=(1, 2, 2)),
        conv2dPlus1d(filters=128, kernel_size=(3, 3, 3), padding='same', activation='relu'),
        layers.MaxPooling3D(pool_size=(1, 2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(1, activation='sigmoid')  # For binary classification
    ])
    return model

In [48]:
input_shape = (10, 64, 64, 3)
model = build_model(input_shape = input_shape)
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [49]:
model.summary()

Model: "sequential_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_plus1d_12 (conv2dPlus1d) │ (None, 10, 64, 64, 32) │         4,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_9 (MaxPooling3D)  │ (None, 10, 32, 32, 32) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_plus1d_13 (conv2dPlus1d) │ (None, 10, 32, 32, 64) │        30,848 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_10 (MaxPooling3D) │ (None, 10, 16, 16, 64) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_plus1d_14 (conv2dPlus1d) │ (None, 10, 16, 16,     │       123,136 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_11 (MaxPooling3D) │ (None, 10, 8, 8, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 81920)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │    10,485,888 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,644,001 (40.60 MB)

 Trainable params: 10,644,001 (40.60 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(train_dataGen, epochs=5, validation_data=test_datagen)

Epoch 1/5
    2/39569 ━━━━━━━━━━━━━━━━━━━━ 1315:43:03 120s/step - accuracy: 0.0234 - loss: -5.2211